In [24]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

df = pd.read_csv('jeopardy.csv')
#print(df.head(5))
#print(len(df))

C:\Users\keluth\AppData\Local\Temp/ipykernel_1704/1892886147.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [30]:
#print(df.dtypes) ## check dtypes
#print(df.columns) ## all column names have space at start of name
#df.columns=['Show Number', 'Air Date', 'Round', 'Category', 'Value', 'Question', 'Answer'] ## rename columns removing the leading space
df = df.rename(columns={'Show Number':'Show Number', 
                        ' Air Date':'Air Date', 
                        ' Round':'Round', 
                        ' Category':'Category', 
                        ' Value':'Value', 
                        ' Question':'Question', 
                        ' Answer':'Answer'}) ## safer way to rename
print(df.columns) ## check columns

Index(['Show Number', 'Air Date', 'Round', 'Category', 'Value', 'Question',
       'Answer'],
      dtype='object')


In [31]:
#print(df.Question)

# filter for specific words
def word_find(data_frame, words):
    filter = lambda x: all(word.lower() in x.lower() for word in words)
    return data_frame.loc[data_frame.Question.apply(filter)]

filtered = word_find(df, ["king", "Henry VIII"])

#print(filtered.Question)
#print(len(filtered))

In [61]:
#print(df.Value)

# remove unwanted characters from series in column
df['Float_Value'] = df.Value.replace('[\$]', '', regex=True)
df['Float_Value'] = df.Float_Value.replace('[\,]', '', regex=True)
#print(len(df.Value))
#print(df.Float_Value) ## check that replace worked

# replace na values (here = none) with 0
df['Float_Value'] = df.Float_Value.replace('[None]', '0', regex=True)
#print(df.Float_Value) ## check that replace worked

# convert values in column to int/float
df['Float_Value'] = pd.to_numeric(df.Float_Value)
#print(df.dtypes) ## conversion worked
#print(df['Float_Value'].mean()) ## check mean

# this can all be done in 1 line of code with the following:
#jeopardy_data["Float Value"] = jeopardy_data["Value"].apply(lambda x: float(x[1:].replace(',','')) if x != "None" else 0)

In [62]:
# find rows with 'king'
filtered = word_find(df, ['king'])

# print mean value of questions containing 'king'
print(filtered['Float_Value'].mean())

771.8833850722094


In [63]:
# create function to determine number of unique answer scores
def n_unique(data):
    return data['Answer'].value_counts()

# print number of unique answer scores associated with a filtered word
print(n_unique(filtered))

Henry VIII                   55
Solomon                      35
Richard III                  33
Louis XIV                    31
David                        30
                             ..
cardiac (in card I acted)    1 
Henderson                    1 
Computer                     1 
Indians                      1 
work                         1 
Name: Answer, Length: 5268, dtype: int64


In [64]:
# identify counts meeting specific criteria
# How many questions in the 1990s contained the word 'computer'?
filtered_data_90s = df[df['Air Date'].str.startswith('199') & df['Question'].str.contains('computer')]
print(len(filtered_data_90s))
# How many questions in the 2000s contained the word 'computer'?
filtered_data_00s = df[df['Air Date'].str.startswith('200') & df['Question'].str.contains('computer')]
print(len(filtered_data_00s))

90
244


In [68]:
#print(df.head(40))

# compare rounds and categories by counting the number of questions of each catregory based on round
round_cat = df.groupby(['Round', 'Category']).Question.count().reset_index()
#print(round_cat)

# calculate the number of questions in 'literature' category based on round and return round with highest count
maxcount = round_cat[round_cat.Category == 'LITERATURE'].Question.max()
print(maxcount)

# print the round (and count) countaining the greatest number of questions in the 'literature' category
print(round_cat[round_cat.Question == maxcount])

381
                 Round    Category  Question
7495  Double Jeopardy!  LITERATURE  381     
